FIRST MODEL

In [5]:
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
from networks import LinearRegressionModel


# Importing the pt files that where created in data_processing.ipynb 
# to avoid separating data again
x_train, y_train = torch.load('train_data.pt')
x_val, y_val = torch.load('val_data.pt')

train_dataset = TensorDataset(x_train, y_train)
val_dataset = TensorDataset(x_val, y_val)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

model = LinearRegressionModel(x_train.shape[1], 1)
optimizer = optim.SGD(model.parameters(), lr=0.001)
criterion = nn.MSELoss()


train_losses = []
val_losses = []

# This model was fast enough to be able to have 120 epochs
# training in only 1 minute

num_epochs = 120
for epoch in range(num_epochs):
    model.train()
    total_train_loss = 0
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()

    train_losses.append(total_train_loss / len(train_loader))

    model.eval()
    total_val_loss = 0
    with torch.no_grad():
        for inputs, targets in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            total_val_loss += loss.item()


    val_losses.append(total_val_loss / len(val_loader))

    print(f'Epoch {epoch+1}, Training Loss: {train_losses[-1]:.3f}, Validation Loss: {val_losses[-1]:.3f}')
    
    with torch.no_grad():
            inputs, actual_ages = next(iter(val_loader))
            predicted_ages = model(inputs).squeeze()

            actual_ages = actual_ages[:15].numpy()
            predicted_ages = predicted_ages[:15].numpy()




torch.save({
    'train_losses': train_losses,
    'val_losses': val_losses,
    'actual_ages': actual_ages,  # Assuming actual_ages is already a numpy array
    'predicted_ages': predicted_ages  # Assuming predicted_ages is already a numpy array
}, 'plot_data.pt')


Epoch 1, Training Loss: 191.047, Validation Loss: 61.578
Epoch 2, Training Loss: 58.700, Validation Loss: 52.876
Epoch 3, Training Loss: 52.745, Validation Loss: 48.565
Epoch 4, Training Loss: 48.835, Validation Loss: 45.431
Epoch 5, Training Loss: 45.931, Validation Loss: 43.026
Epoch 6, Training Loss: 43.602, Validation Loss: 40.960
Epoch 7, Training Loss: 41.575, Validation Loss: 39.287
Epoch 8, Training Loss: 39.851, Validation Loss: 37.658
Epoch 9, Training Loss: 38.348, Validation Loss: 36.228
Epoch 10, Training Loss: 36.946, Validation Loss: 35.000
Epoch 11, Training Loss: 35.668, Validation Loss: 33.772
Epoch 12, Training Loss: 34.515, Validation Loss: 32.688
Epoch 13, Training Loss: 33.457, Validation Loss: 31.639
Epoch 14, Training Loss: 32.516, Validation Loss: 30.680
Epoch 15, Training Loss: 31.555, Validation Loss: 29.881
Epoch 16, Training Loss: 30.747, Validation Loss: 29.008
Epoch 17, Training Loss: 29.932, Validation Loss: 28.262
Epoch 18, Training Loss: 29.198, Valida

SECOND MODEL

In [2]:
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader
from networks import AgePredictionCNN
from dataloader import ImageDataset
from torchvision import transforms
from sklearn.model_selection import train_test_split
import os
import matplotlib.pyplot as plt


transformations = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


img_dir = 'images/images'                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         
all_images = [img for img in os.listdir(img_dir) if img.endswith('.jpg')]
train_files, val_files = train_test_split(all_images, test_size=0.2, random_state=42)


train_dataset = ImageDataset(train_files, img_dir, transformations)
val_dataset = ImageDataset(val_files, img_dir, transformations)


train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AgePredictionCNN().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

train_losses = []
val_losses = []
actual_ages = []
predicted_ages = []
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

num_epochs = 3                                                                                                                                                                                                                                                                         
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs).squeeze()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    avg_train_loss = train_loss / len(train_loader)
    train_losses.append(avg_train_loss)

    model.eval()
    val_loss = 0.0
    ages_recorded = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs).squeeze()
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            if ages_recorded<15:
                batch_size = labels.size(0)
                batch_take = min(15 - ages_recorded, batch_size)

                actual_ages.extend(labels[:batch_take].cpu().numpy())
                predicted_ages.extend(outputs[:batch_take].cpu().numpy())

                ages_recorded += batch_take
            if ages_recorded >=15:
                break


    avg_val_loss = val_loss / len(val_loader)
    val_losses.append(avg_val_loss)

    print(f'Epoch {epoch+1}, Train Loss: {avg_train_loss}, Val Loss: {avg_val_loss}')
        

torch.save({
    'train_losses2': train_losses,
    'val_losses2': val_losses,
    'actual_ages2': actual_ages, 
    'predicted_ages2': predicted_ages  
}, 'plot_data2.pt')


torch.save(model.state_dict(), 'age_prediction_model.pth')


Epoch 1, Train Loss: 249.51276957707773, Val Loss: 1.866197509765625
Epoch 2, Train Loss: 139.79027521409571, Val Loss: 1.7847794596354167
Epoch 3, Train Loss: 108.91323623914109, Val Loss: 1.6594936116536458


THIRD MODEL

In [4]:
import matplotlib.pyplot as plt
from torchvision import transforms
from torch.utils.data import DataLoader
from dataloader import ThirdImageDataset
from networks import ThirdCNN
import torch.optim as optim
import torch.nn as nn
import torch


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


dataset = ThirdImageDataset(csv_file='UTKFaceAugmented.csv', img_dir='images/images', transform=transform)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64)


num_features = len(dataset[0][1])  
model = ThirdCNN(num_features, 1)  
model.to(device)


criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)

scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)


num_epochs = 2
train_losses = []
val_losses = []
best_val_loss = float('inf')

#early stopping to prevent overfitting
patience, max_patience = 0, 5 

for epoch in range(num_epochs):
    model.train()
    total_train_loss = 0
    for images, features, labels in train_loader:
        images, features, labels = images.to(device), features.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images, features)
        loss = criterion(outputs.squeeze(), labels)
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()
    avg_train_loss = total_train_loss / len(train_loader)
    train_losses.append(avg_train_loss)

    model.eval()
    total_val_loss = 0
    for images, features, labels in val_loader:
        images, features, labels = images.to(device), features.to(device), labels.to(device)
        outputs = model(images, features)
        loss = criterion(outputs.squeeze(), labels)
        total_val_loss += loss.item()
    avg_val_loss = total_val_loss / len(val_loader)
    val_losses.append(avg_val_loss)

    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        patience = 0
        torch.save(model.state_dict(), 'best_age_prediction_model.pth')
    else:
        patience += 1
        if patience > max_patience:
            print("Early stopping triggered.")
            break

    scheduler.step()  

    print(f'Epoch {epoch + 1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}')


model.load_state_dict(torch.load('best_age_prediction_model.pth'))
model.eval()
actual_ages = []
predicted_ages = []

# Evaluation loop where we collect the predictions
with torch.no_grad():
    for images, features, labels in val_loader:
        images, features, labels = images.to(device), features.to(device), labels.to(device)
        outputs = model(images, features).squeeze()  # Make sure to squeeze the outputs if they contain an extra dimension
        predicted_ages.extend(outputs.cpu().numpy().tolist())  # Extend the list with the model's predictions
        actual_ages.extend(labels.cpu().numpy().tolist())  # Extend the list with the actual labels

# Save the losses and predictions
torch.save({
    'train_losses3': train_losses,
    'val_losses3': val_losses,
    'actual_ages3': actual_ages, 
    'predicted_ages3': predicted_ages  
}, 'plot_data3.pt')



#ChatGPT helped with part of the implementation of these models  


Epoch 1/2, Train Loss: 3519929.4921, Val Loss: 1508.5648
Epoch 2/2, Train Loss: 1498.0348, Val Loss: 1508.5648
